In [2]:
import numpy as np
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests

In [2]:
merged = pd.read_csv('merged_data.csv', index_col = 'Title')

In [61]:
movie_data = pd.read_csv('merged_data.csv') 
movie_data = movie_data[['Title','WikiID','Genres', 'Summary','pos','neg', 'compound', 'neu']]

movie_to_id = {}
for t, title in enumerate(movie_data['Title']):
    wiki = movie_data['WikiID'][t]
    movie_to_id[title] = wiki

id_to_index = {}
for i, wikiid in enumerate(movie_data['WikiID']):
    id_to_index[wikiid] = i

def get_genres(movie):
    wiki_id = movie_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    genre = movie_data['Genres'][ind]
    return genre

def get_summary(movie):
    wiki_id = movie_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    summary = movie_data['Summary'][ind]
    return summary

def get_pos(movie):
    wiki_id = movie_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    pos = movie_data['pos'][ind]
    return pos

def get_neg(movie):
    wiki_id = movie_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    neg = movie_data['neg'][ind]
    return neg

def get_compound(movie):
    wiki_id = movie_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    comp = movie_data['compound'][ind]
    return comp

def get_neu(movie):
    wiki_id = movie_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    neu = movie_data['neu'][ind]
    return neu
    

In [56]:
movie_to_id['Ghosts of Mars']

975900

In [69]:
 def get_sim(mov1, mov2):
#  need to fix so that the cosine similarity is calculated between the user input movie and the top250
    vectorizer = TfidfVectorizer()
    sum1 = get_summary(mov1)
    sum2 = get_summary(mov2)
    doc = [sum1, sum2]
    tfidf = vectorizer.fit_transform(doc)
    return cosine_similarity(tfidf)[0][1]

In [52]:
get_sim('Ghosts of Mars', 'White Of The Eye')

A series of murders of rich young women throughout Arizona bear distinctive signatures of a serial killer. Clues lead Detective Charles Mendoza to visit Paul White, a sound expert installing hi-fi systems in wealthy people's homes. His special talent is to make a noise which echoes through the air cavities in his head and shows him where the sound of the speakers should come from and echo in the room. He is married to Joan, whom, ten years earlier, he had seduced away from Mike DeSantos, her then current boyfriend. Joan is questioned by Mendoza, but does not believe his insinuations that her husband is somehow involved in the murders. Various flashbacks show Joan's previous relationship to Mike and later explain how it came to be that he abandoned her. The couple met Paul and befriended him. At Mike's suggestion, he and Mike go on a deer hunting trip together. Paul shoots a deer and brutally mutilates it, demonstrating his sick fascination with killing. This is partly intended to scare

0.4805725814816539

In [72]:
def compare_sim(mov1):
    cosine_sim = {}
    for title in movie_data['Title']:
        if title == mov1:
            value = 1
        else:
            value = get_sim(mov1, title)
        cosine_sim[title] = value
    return cosine_sim

In [73]:
compare_sim('Ghosts of Mars')

{'Ghosts of Mars': 1,
 'White Of The Eye': 0.4805725814816539,
 'A Woman in Flames': 0.5313727096641044,
 "The Sorcerer's Apprentice": 0.5218574261467509,
 'Little city': 0.35943418338138117,
 'Henry V': 0.687045724266953,
 'Aaah Belinda': 0.414714522681395,
 'The Mechanical Monsters': 0.7071802928991542,
 'Mary Poppins': 0.6833357642187516,
 'Die Fahne von Kriwoj Rog': 0.6522142523130408,
 'White on Rice': 0.17887899647107108,
 'Anbu Thozhi': 0.3742385311822046,
 'Baby Boy': 0.4069349562181141,
 'Bindiya Chamkegi': 0.2903529151337284,
 'Vandanam': 0.5776433327125212,
 'Anokha Rishta': 0.5169684360604677,
 'Karayilekku Oru Kadal Dooram': 0.3416284829605678,
 'Siam Sunset': 0.35249517559609267,
 'Kausthubham': 0.38941583696443405,
 'Archie: To Riverdale and Back Again': 0.4639034091469757,
 'Troops': 0.5652188190142152,
 'Daddy and Them': 0.09683215394819668,
 'The Gods Must Be Crazy': 0.667209390122343,
 'Rudo y Cursi': 0.5847254577328125,
 'Kinjite: Forbidden Subjects': 0.475279777577

In [105]:
def cleanjson(result):
    title = result[result.find("Title")+8:result.find("Year")-3]
    plot = result[result.find("Plot")+7:result.find("Language")-3]
    review_imdb = float(result[result.find('"Internet Movie Database","Value":"')+35:result.find('Source":"Rotten Tomatoes"')-8])
    review_rotten = float(result[result.find('Source":"Rotten Tomatoes","Value":')+35: result.find('},{"Source":"Metacritic"')-2])
    return [title,plot,review_imdb,review_rotten]

def response(result):
    text = result[result.find('"Response":' )+12:]
    return text.find('True') >-1
        



In [107]:
def find_movie(movie):
    output = []
    #find the movie plot summary using IMDB API
    token = 'ce887dbd'
    query = "http://www.omdbapi.com/?apikey=" + token + "&t=" + movie
    #query1 = "http://www.omdbapi.com/?apikey=" + token + "&s=" + movie
    params = {"r": "json", "plot": "full"}
    result = requests.get(query, params)
    if response(result.text):
        json = cleanjson(result.text)
        plot = json[1]
        title = json[0]
        review_imdb = json[2]
        review_rotten = json[3]
        output.append('Plot of the movie entered : ' + str(plot))
    else:
        output.append("We did not find the movie you searched for. Did you spell it correctly?")




    return output

x = find_movie('Inception')
print(x)


["Plot of the movie entered : Dom Cobb is a skilled thief, the absolute best in the dangerous art of extraction, stealing valuable secrets from deep within the subconscious during the dream state, when the mind is at its most vulnerable. Cobb's rare ability has made him a coveted player in this treacherous new world of corporate espionage, but it has also made him an international fugitive and cost him everything he has ever loved. Now Cobb is being offered a chance at redemption. One last job could give him his life back but only if he can accomplish the impossible - inception. Instead of the perfect heist, Cobb and his team of specialists have to pull off the reverse: their task is not to steal an idea but to plant one. If they succeed, it could be the perfect crime. But no amount of careful planning or expertise can prepare the team for the dangerous enemy that seems to predict their every move. An enemy that only Cobb could have seen coming."]
